In [1]:
import pyspiel
import open_spiel
from open_spiel.python.algorithms import external_sampling_mccfr
from open_spiel.python.algorithms import cfr
import random
import pickle
from datetime import datetime
import random
# print(pyspiel.__version__``)
# print(dir(open_spiel.python.Dict))
!pip show open_spiel

Optional module pokerkit_wrapper was not importable: No module named 'pokerkit'


Name: open_spiel
Version: 1.6.179.25
Summary: A Framework for Reinforcement Learning in Games
Home-page: https://github.com/deepmind/open_spiel
Author: The OpenSpiel authors
Author-email: open_spiel@google.com
License: Apache 2.0
Location: /workspaces/codespaces-blank/ovenv/lib/python3.12/site-packages
Requires: absl-py, attrs, ml-collections, numpy, pip, scipy
Required-by: 


In [3]:
VALUES = {"players": 2, "numdice": 1, "dice_sides": 6, "recall_length": 3}
game = pyspiel.load_game("perudo_ir", VALUES)
VALUES["liarbid"] = game.max_history_length() - 1 - VALUES["numdice"] * VALUES["players"]
solver = external_sampling_mccfr.ExternalSamplingSolver(game)
print(game.max_history_length() - 1 - VALUES["numdice"] * VALUES["players"])

12


In [4]:
def get_timestamp():
    return datetime.now().strftime("%Y-%m-%d-%H:%M")
print(get_timestamp())
def save(solver):
    with open(f"models/mccfr_{get_timestamp()}.pkl", "wb") as file:
        pickle.dump(solver, file)
def load_solver(filepath):
    with open(filepath, "rb") as file:
        solver = pickle.load(file)
    return solver

2026-01-27-21:41


In [5]:
# solver = load_solver("models/2cubes_mccfr_2026-01-27-09:59.pkl")
solver = load_solver("models/mccfr_2026-01-27-09:22.pkl")

In [6]:
class Act:
    def __init__(self, x=None):
        if x is None:
            x = input()
        if type(x) == str:
            self.strbid = x
            l = x.split("-")
            if len(l) != 2 and x == 'd':
                self.strbid = "Stop!"
                self.intbid = -1
            elif len(l) != 2:
                self.intbid = VALUES["liarbid"]
                self.strbid = "Liar!"
            else:
                self.intbid = (int(l[0])-1) * VALUES["dice_sides"] + int(l[1]) - 1
        else:
            self.intbid = int(x)
            self.strbid = f"{x // VALUES["dice_sides"] + 1}-{x % VALUES["dice_sides"] + 1}"
            if self.intbid == VALUES["liarbid"]:
                self.strbid = "Liar!"
    
    @staticmethod
    def solver_act(solver, state):
        d = solver.average_policy().action_probabilities(state)
        values = []
        weights = []
        for x in d:
            values.append(x)
            weights.append(d[x])
        return Act(random.choices(values, weights=weights, k=1)[0])

    def __str__(self):
        return self.strbid
    
def print_cubes(state, player_id):
    cubes = []
    for i in range(VALUES["numdice"]):
        cubes.append(state.history()[VALUES["numdice"] * player_id + i] + 1)
    print(*cubes)


In [7]:
ans = None
wins = 0
losses = 0
numdice = VALUES["numdice"]
cont = True
while cont:
    print("----------------------------------------------")
    print("New game")
    player_id = random.choice([0, 1])
    if player_id == 0:
        print("You go first")
    else:
        print("You go second")
    state = game.new_initial_state()

    while state.is_chance_node():
        values, weights = zip(*state.chance_outcomes())
        action = random.choices(values, weights=weights, k=1)[0]
        state.apply_action(action)
    print(f"Your dice are:", end = ' ')
    print_cubes(state, player_id)

    while not state.is_terminal():
        if state.current_player() == player_id:
            print("Your bid is:", end = ' ')
            # print(state.legal_actions())
            act = Act() # reads
            print(act)
            if act.intbid == -1:
                cont = False
                break
            state.apply_action(act.intbid)
        else:
            act = Act.solver_act(solver, state)
            print(act)
            state.apply_action(act.intbid)
    if not cont:
        break
    print(f"His cube were:")
    print_cubes(state, 1 - player_id)
    # print(state.rewards())
    if state.rewards()[player_id] == 1:
        print("You won!")
        wins += 1
    else:
        print("You lost!")
        losses += 1
    print(f"History: {state.history_str()}")

----------------------------------------------
New game
You go first
Your dice are: 5
Your bid is: 

1-2
1-1
Your bid is: Liar!
His cube were:
2
You won!
History: 4, 1, 1, 0, 12
----------------------------------------------
New game
You go second
Your dice are: 3
1-5
Your bid is: Liar!
His cube were:
1
You lost!
History: 0, 2, 4, 12
----------------------------------------------
New game
You go first
Your dice are: 2
Your bid is: 1-2
1-1
Your bid is: Liar!
His cube were:
1
You lost!
History: 1, 0, 1, 0, 12
----------------------------------------------
New game
You go first
Your dice are: 5
Your bid is: 1-3
1-6
Your bid is: 1-1
2-2
Your bid is: Liar!
His cube were:
3
You won!
History: 4, 2, 2, 5, 0, 7, 12
----------------------------------------------
New game
You go second
Your dice are: 1
2-5
Your bid is: 1-1
Liar!
His cube were:
2
You won!
History: 1, 0, 10, 0, 12
----------------------------------------------
New game
You go second
Your dice are: 5
1-6
Your bid is: Liar!
His cube were:
2
You won!
History: 1, 4, 5, 12
----------------------------------------------
New game
You go 

: 

In [26]:
print(wins, losses)

6 6


In [10]:
print(Act(12))
print(Act("l"))

3-1
Liar!


In [20]:
state = game.new_initial_state()
state.apply_action(1)
state.apply_action(2)
probs = solver.average_policy().action_probabilities(state)
for i in range(12):
    print(round(probs[i], 2), end = ' ')


0.01 0.17 0.03 0.15 0.32 0.32 0.0 0.0 0.0 0.0 0.0 0.0 

In [ ]:
for i in range(6):
    state = game.new_initial_state()
    state.apply_action(i)
    state.apply_action(i)
    probs = solver.average_policy().action_probabilities(state)
    print(f"{i+1}: ", end='')
    for i in range(7):
        print(round(probs[i], 2), end = ' ')
    print()